In [ ]:
%env  WORKDIR=/tmp/vault

In [ ]:
%%bash
cat $WORKDIR/VAULT_TOKEN.TXT
cat $WORKDIR/VAULT_ADDR.TXT

Update the values of the enviromental values of the following cells based on the output of the previous cell

In [ ]:
%env VAULT_ADDR=https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200
%env VAULT_TOKEN=hvs.QbtInWOOYqrZJnMrDmcaArKs

## Step 1. Install MongoDB

In [ ]:
! kubectl apply -f mongodb_deploy.yaml

# Step 2. Configure Vault

In [ ]:
# Enable Secret Engine
! vault secrets enable -path=mongodb database

In [ ]:
%%bash
export MONGO_ADDR=$(kubectl get svc -n mongodb mongodb -o json | jq  -r '.status.loadBalancer.ingress[0].ip')

# Create Connection
vault write mongodb/config/mongodb \
    plugin_name=mongodb-database-plugin \
    allowed_roles="*"  connection_url="mongodb://{{username}}:{{password}}@${MONGO_ADDR}:27017/admin?tls=false" \
    username="mdbadmin" \
    password="Passw0rd123!"

In [ ]:
%%bash
# Create Role
vault write mongodb/roles/mongo-test \
    db_name=mongodb \
    creation_statements='{ "db": "admin", "roles": [{ "role": "readWrite" }, {"role": "read", "db": "foo"}] }' \
    default_ttl="1h" \
    max_ttl="24h"



In [ ]:
# Create Creds
! vault read mongodb/creds/mongo-test

# Clean UP

In [ ]:
%%bash
vault lease revoke -prefix mongodb/creds/*
vault secrets disable mongodb
kubectl delete -f mongodb_deploy.yaml
